$$(\ddot{r} - r\dot{\phi}^2)\,\hat{\bm{r}} + (r\ddot{\phi} + 2\dot{r}\dot{\phi})\,\hat{\bm{\phi}}=g\hat{r}-\frac{\tau}{m|\dot{r}|}[\dot{r}\hat{r}=r\hat{\phi}\dot{\phi}]$$
$$\dot{m}=\frac{\Tau}{V_e}$$
$$\dot{r}(0)=v_0, \quad \phi$$

In [5]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root

# ------------------ parameters ------------------
mu   = 4.9048695e12      # GM of body [m^3/s^2]  (change as needed)
T    = 1.0e5             # thrust [N]
Ve   = 3.0e3             # exhaust velocity [m/s]

# initial/final conditions YOU care about
r0    = 1.9e6            # initial radius [m]
vphi0 = 1.6e3 / r0       # initial angular rate [rad/s] (example)
m0    = 15000.0          # initial mass [kg]

rf_target = 1.7e6        # desired final radius [m]
vf_target = 0.0          # desired final radial velocity [m/s]

# ------------------ equations of motion ------------------
def eom(t, y):
    """
    y = [r, vr, phi, vphi, m]
    r     : radius
    vr    : radial velocity
    phi   : angle
    vphi  : angular rate (dphi/dt)
    m     : mass
    """
    r, vr, phi, vphi, m = y

    # gravity (central field)
    g_r = -mu / r**2          # radial acceleration from gravity

    # thrust & mass flow
    if m <= 0:
        thrust = 0.0
        mdot   = 0.0
    else:
        thrust = T
        mdot   = -T / Ve

    # velocity components (for direction of thrust)
    v_r       = vr
    v_tan     = r * vphi
    speed     = np.hypot(v_r, v_tan)

    if speed == 0.0 or thrust == 0.0:
        a_Tr   = 0.0
        a_Ttan = 0.0
    else:
        aT     = thrust / m
        # thrust opposite velocity
        a_Tr   = -aT * v_r   / speed      # radial component
        a_Ttan = -aT * v_tan / speed      # tangential (linear) component

    # convert tangential linear accel -> angular equation
    rdot     = vr
    vrdot    = r * vphi**2 + g_r + a_Tr
    phidot   = vphi
    vphidot  = (a_Ttan - 2.0 * vr * vphi) / r

    return [rdot, vrdot, phidot, vphidot, mdot]


In [6]:
def terminal_error(params):
    """
    params[0] = tf   (burn duration guess)
    params[1] = vr0  (initial radial velocity guess)

    Returns [r(tf) - rf_target, vr(tf) - vf_target]
    """
    tf, vr0 = params

    # initial state: phi(0)=0 w.l.o.g.
    y0 = [r0, vr0, 0.0, vphi0, m0]

    sol = solve_ivp(
        eom,
        t_span=(0.0, tf),
        y0=y0,
        rtol=1e-8,
        atol=1e-8,
        dense_output=False,
    )

    r_f, vr_f = sol.y[0, -1], sol.y[1, -1]

    return np.array([r_f - rf_target, vr_f - vf_target])


In [ ]:
# initial guesses: [tf_guess, vr0_guess]
params0 = np.array([200.0, 0.0])   # 200 s burn, ~0 initial radial speed

sol_root = root(terminal_error, params0)

if not sol_root.success:
    print("Root find failed:", sol_root.message)
else:
    tf_sol, vr0_sol = sol_root.x
    print(f"Solved tf  = {tf_sol:.3f} s")
    print(f"Solved vr0 = {vr0_sol:.3f} m/s")

    # integrate once more with the solved parameters to get full trajectory
    y0 = [r0, vr0_sol, 0.0, vphi0, m0]
    sol = solve_ivp(
        eom,
        t_span=(0.0, tf_sol),
        y0=y0,
        rtol=1e-8,
        atol=1e-8,
        dense_output=True,
    )

    r_f, vr_f, phi_f, vphi_f, m_f = sol.y[:, -1]
    print("Final state at tf:")
    print(f"  r(tf)  = {r_f:.3f} m")
    print(f"  vr(tf) = {vr_f:.3f} m/s")
    print(f"  phi(tf)= {phi_f:.6f} rad")
    print(f"  vphi(tf)= {vphi_f:.6e} rad/s")
    print(f"  m(tf)  = {m_f:.3f} kg")       

KeyboardInterrupt: 

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root

# -------------- problem parameters (edit these) --------------
mu   = 4.9048695e12   # GM of body [m^3/s^2] (example: Moon)
T    = 1.0e5          # thrust [N]
Ve   = 3.0e3          # exhaust velocity [m/s]

# Your boundary data
r0       = 1.9e6      # initial radius [m]
vphi0    = 1.6e3 / r0 # initial angular rate [rad/s] (example)
m0       = 15000.0    # initial mass [kg]
rf_target = 1.7e6     # desired final radius [m]
vf_target = 0.0       # desired final radial velocity [m/s]

# --- search bounds for unknowns (YOU SHOULD TUNE THESE) ---
tf_min, tf_max       = 10.0, 2000.0        # seconds
vr0_min, vr0_max     = -2000.0, 2000.0     # m/s

# ------------------- equations of motion -------------------
def eom(t, y):
    """
    y = [r, vr, phi, vphi, m]
    """
    r, vr, phi, vphi, m = y

    # gravity
    g_r = -mu / r**2

    # thrust & mass flow
    if m <= 0:
        thrust = 0.0
        mdot   = 0.0
    else:
        thrust = T
        mdot   = -T / Ve

    # velocity components
    v_r   = vr
    v_tan = r * vphi
    speed = np.hypot(v_r, v_tan)

    if speed == 0.0 or thrust == 0.0:
        a_Tr   = 0.0
        a_Ttan = 0.0
    else:
        aT     = thrust / m
        a_Tr   = -aT * v_r   / speed      # opposite velocity
        a_Ttan = -aT * v_tan / speed

    # convert tangential accel to angular equation
    rdot    = vr
    vrdot   = r * vphi**2 + g_r + a_Tr
    phidot  = vphi
    vphidot = (a_Ttan - 2.0 * vr * vphi) / r

    return [rdot, vrdot, phidot, vphidot, mdot]

# ---------- reparametrization helpers (bounds) --------------
def unpack_params(u):
    """
    u[0], u[1] are unconstrained
    map them into bounded tf, vr0
    """
    u_tf, u_vr = u

    # logistic mapping -> (tf_min, tf_max)
    tf  = tf_min + (tf_max - tf_min) / (1.0 + np.exp(-u_tf))
    vr0 = vr0_min + (vr0_max - vr0_min) / (1.0 + np.exp(-u_vr))

    return tf, vr0

# -------------- terminal-error function --------------------
def terminal_error(u):
    """
    u[0], u[1] -> unconstrained variables
    phys params: tf, vr0
    returns [r(tf) - rf_target, vr(tf) - vf_target]
    """
    tf, vr0 = unpack_params(u)

    y0 = [r0, vr0, 0.0, vphi0, m0]

    sol = solve_ivp(
        eom,
        t_span=(0.0, tf),
        y0=y0,
        rtol=1e-7,
        atol=1e-7,
        dense_output=False,
    )

    r_f  = sol.y[0, -1]
    vr_f = sol.y[1, -1]

    err = np.array([r_f - rf_target, vr_f - vf_target])
    # optional: print to watch convergence
    print(f"tf ~ {tf:7.1f} s, vr0 ~ {vr0:7.1f} m/s -> "
          f"dr = {err[0]:8.1f} m, dvr = {err[1]:7.2f} m/s")
    return err

# ------------------------ solve ----------------------------
# initial guess in unconstrained space
u0 = np.array([0.0, 0.0])   # roughly middle of each range

sol_root = root(
    terminal_error,
    u0,
    method='hybr',
    options={'maxfev': 30, 'xtol': 1e-4}
)

if not sol_root.success:
    print("\nRoot find did NOT converge in the chosen ranges.")
    print("Message:", sol_root.message)
else:
    tf_sol, vr0_sol = unpack_params(sol_root.x)
    print("\nSUCCESS:")
    print(f"  tf  = {tf_sol:.3f} s")
    print(f"  vr0 = {vr0_sol:.3f} m/s")

    # integrate once more to get full trajectory
    y0 = [r0, vr0_sol, 0.0, vphi0, m0]
    sol = solve_ivp(
        eom, (0.0, tf_sol), y0, rtol=1e-8, atol=1e-8, dense_output=True
    )
    r_f, vr_f, phi_f, vphi_f, m_f = sol.y[:, -1]
    print("Final state at tf:")
    print(f"  r(tf)   = {r_f:.3f} m")
    print(f"  vr(tf)  = {vr_f:.3f} m/s")
    print(f"  phi(tf) = {phi_f:.6f} rad")
    print(f"  vphi(tf)= {vphi_f:.6e} rad/s")
    print(f"  m(tf)   = {m_f:.3f} kg")


KeyboardInterrupt: 

In [3]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root

# -------------------- PHYSICAL PARAMETERS --------------------
mu   = 4.9048695e12   # GM of body [m^3/s^2], example: Moon. Change as needed.
T    = 1.0e5          # thrust [N]
Ve   = 3.0e3          # exhaust velocity [m/s]

# -------------------- YOUR KNOWN BCs -------------------------
# Initial
vphi0 = 0.0           # dphi/dt at t=0 [rad/s]  <-- SET THIS
m0    = 15000.0       # mass at t=0 [kg]        <-- SET THIS

# Final (desired at t = tf)
rf_target    = 1.7e6      # r(tf) [m]           <-- SET THIS
phif_target  = 0.0      # phi(tf) [rad]       <-- SET THIS
vrf_target   = 0.0        # dr/dt(tf) [m/s]     <-- SET THIS
vphif_target = 0.0        # dphi/dt(tf) [rad/s] <-- SET THIS

# ----------------- SEARCH RANGES FOR UNKNOWNs ----------------
# We’ll solve for r0, vr0, phi0, tf, but keep them bounded for sanity.
r0_min, r0_max       = 0.5 * rf_target, 2.0 * rf_target
vr0_min, vr0_max     = -2000.0, 2000.0         # m/s, tweak to taste
phi0_min, phi0_max   = -np.pi, np.pi
tf_min, tf_max       = 10.0, 3000.0           # seconds (rough bounds)

# --------------- EQUATIONS OF MOTION (POLAR) -----------------
def eom(t, y):
    """
    y = [r, vr, phi, vphi, m]
    """
    r, vr, phi, vphi, m = y

    # Gravity term: central field. If you want constant g instead, replace this
    # with something like g_r = -g0
    g_r = -mu / r**2

    # Thrust & mass flow
    if m <= 0:
        thrust = 0.0
        mdot   = 0.0
    else:
        thrust = T
        mdot   = -T / Ve

    # Velocity components
    v_r   = vr
    v_tan = r * vphi
    speed = np.hypot(v_r, v_tan)

    if thrust == 0.0 or speed == 0.0:
        a_Tr   = 0.0
        a_Ttan = 0.0
    else:
        aT     = thrust / m
        # Suicide-burn-ish: thrust opposite velocity
        a_Tr   = -aT * v_r   / speed   # radial component
        a_Ttan = -aT * v_tan / speed   # tangential (linear) component

    # Dynamics in polar form
    rdot    = vr
    vrdot   = r * vphi**2 + g_r + a_Tr
    phidot  = vphi
    vphidot = (a_Ttan - 2.0 * vr * vphi) / r

    return [rdot, vrdot, phidot, vphidot, mdot]

# --------- MAP UNCONSTRAINED u -> PHYSICAL (bounded) ---------
def logistic_to_range(u, a, b):
    """Map (-inf, inf) -> (a, b) smoothly."""
    return a + (b - a) / (1.0 + np.exp(-u))

def unpack_params(u):
    """
    u: length-4 unconstrained vector
        u[0] -> r0   in [r0_min, r0_max]
        u[1] -> vr0  in [vr0_min, vr0_max]
        u[2] -> phi0 in [phi0_min, phi0_max]
        u[3] -> tf   in [tf_min, tf_max]
    """
    u_r0, u_vr0, u_phi0, u_tf = u
    r0   = logistic_to_range(u_r0,   r0_min,   r0_max)
    vr0  = logistic_to_range(u_vr0,  vr0_min,  vr0_max)
    phi0 = logistic_to_range(u_phi0, phi0_min, phi0_max)
    tf   = logistic_to_range(u_tf,   tf_min,   tf_max)
    return r0, vr0, phi0, tf

# --------------------- RESIDUAL FUNCTION ---------------------
def terminal_error(u):
    """
    u: unconstrained parameters; we map to (r0, vr0, phi0, tf),
    integrate, and compare final state to desired (rf, phif, vrf, vphif).

    Returns a 4-vector of residuals.
    """
    r0, vr0, phi0, tf = unpack_params(u)

    # initial state, using your known vphi0 and m0
    y0 = [r0, vr0, phi0, vphi0, m0]

    sol = solve_ivp(
        eom,
        t_span=(0.0, tf),
        y0=y0,
        rtol=1e-7,
        atol=1e-7,
        dense_output=False,
    )

    r_f, vr_f, phi_f, vphi_f, m_f = sol.y[:, -1]

    # residuals: want these to be ~0
    res = np.array([
        r_f   - rf_target,
        phi_f - phif_target,
        vr_f  - vrf_target,
        vphi_f - vphif_target,
    ])

    # Optional: print progress so you can see what it's doing
    print(
        f"try r0={r0:8.1f}, vr0={vr0:7.1f}, phi0={phi0:6.3f}, tf={tf:7.1f}"
        f" -> dr={res[0]:8.1f}, dphi={res[1]:7.4f}, dvr={res[2]:7.2f}, dvphi={res[3]:8.3e}"
    )
    return res

# -------------------------- SOLVE ----------------------------
# initial guess in unconstrained space (all zeros = mid-range)
u0 = np.zeros(4)

sol_root = root(
    terminal_error,
    u0,
    method="hybr",
    options={"maxfev": 40, "xtol": 1e-5},
)

if not sol_root.success:
    print("\nRoot-find did NOT converge within the given bounds.")
    print("Message:", sol_root.message)
else:
    r0_sol, vr0_sol, phi0_sol, tf_sol = unpack_params(sol_root.x)
    print("\nSUCCESS!")
    print(f"  r0  = {r0_sol:.3f} m")
    print(f"  vr0 = {vr0_sol:.3f} m/s")
    print(f"  phi0= {phi0_sol:.6f} rad")
    print(f"  tf  = {tf_sol:.3f} s")

    # re-integrate with the solved parameters to get full trajectory
    y0 = [r0_sol, vr0_sol, phi0_sol, vphi0, m0]
    sol = solve_ivp(
        eom,
        (0.0, tf_sol),
        y0,
        rtol=1e-8,
        atol=1e-8,
        dense_output=True,
    )

    r_f, vr_f, phi_f, vphi_f, m_f = sol.y[:, -1]
    print("Final state reached:")
    print(f"  r(tf)   = {r_f:.3f} m   (target {rf_target:.3f})")
    print(f"  vr(tf)  = {vr_f:.3f} m/s (target {vrf_target:.3f})")
    print(f"  phi(tf) = {phi_f:.6f} rad (target {phif_target:.6f})")
    print(f"  vphi(tf)= {vphi_f:.6e} rad/s (target {vphif_target:.6e})")
    print(f"  m(tf)   = {m_f:.3f} kg")


KeyboardInterrupt: 